# The Santa Claus Problem

Integrantes:

Heitor Souza - 744596

Jhonata Querobim - 727342

O Papai Noel está dormindo em sua loja no Polo Norte e só pode ser acordado por todas as 9 renas ou um grupo de 3 elfos:

*   Se as 9 renas chegarem, o Papai Noel acorda.
*   Se 3 elfos precisarem de ajuda, o Papai Noel acorda.
*   Se 3 elfos estão esperando e a última rena chegar, o Papai Noel prioriza as renas.

Utilização da implementação de semáforos POSIX: [Synchronizing Threads with POSIX Semaphores](http://www.csc.villanova.edu/~mdamian/threads/posixsem.html)

In [ ]:
// Solução proposta 
elves = 0
reindeer = 0
santaSem = Semaphore (0)
reindeerSem = Semaphore (0)
elfTex = Semaphore (1)
mutex = Semaphore (1)

In [ ]:
int main() {
    elves = 0
    reindeer = 0
    sem_init(&santaSem, 0, 0);
    sem_init(&reindeerSem, 0, 0);
    sem_init(&elfTex, 0, 1);
    sem_init(&mutex, 0, 1);
}

# Solução do Papai Noel

In [ ]:
// Santa problem solution (Santa), runs in a loop.
santaSem.wait()
mutex.wait()
  if  reindeer  == 9
    prepareSleigh ()
    reindeerSem.signal (9)
    reindeer  -= 9
  else if  elves  == 3
    helpElves ()
mutex.signal ()


*   O Papai Noel acorda e verifica as duas condições e lida com as renas ou os elfos
*   Se houver nove renas esperando, o Papai Noel chama o método de preparar o trenó e, em seguida, sinalizará para as renas 9 vezes, permitindo que chame o método getHitched()
*   Se houver elfos esperando, Papai Noel apenas chama o método helpElves(). Os elfos não precisam esperar pelo Papai Noel; uma vez que eles sinalizam para eles, eles podem invocar o método imediatamente.

In [ ]:
void Santa(){
    // Loop
    while(True){
        sem_wait(&santaSem);
        sem_wait(&mutex);
     
        if (reindeer == 9){
            prepareSleigh ();
            
            for(int i=0; i<9; i++)
                sem_post(&semReindeer);          
        
            reindeer = 0;     
        }

        else if (elves == 3) {
            helpElves();
        }

        sem_post(&mutex);
    }
}

# Solução da Rena

In [ ]:
// Solução do livro
mutex.wait()
    reindeer  += 1
    if  reindeer  == 9:
        santaSem.signal ()
    mutex.signal ()

reindeerSem.wait()
getHitched ()



*   A nona rena sinaliza para o Papai Noel e depois, junto com as outra renas, fica em espera.
*   Quando o Papai Noel sinaliza, todas as renas 
executam getHitched(). 

In [ ]:
//Implementação 
void Reindeer(){
    while (true) {
        sem_wait(&mutex);
        reindeer++;

        if (reindeer == 9)
            sem_post(&santaSem);

        sem_post(&mutex);
        sem_wait(&reindeerSem);
        
        getHitched();
    }
}

# Solução dos Elfos

In [ ]:
elfTex.wait()
mutex.wait()
    elves  += 1
    if  elves  == 3:
        santaSem.signal ()
    else
        elfTex.signal ()
mutex.signal ()

getHelp ()

mutex.wait()
    elves  -= 1
    if  elves  == 0:
        elfTex.signal ()
mutex.signal ()



*   Os dois primeiros elfos liberam o mutex e   o elfTex ao mesmo tempo, mas o último elfo trava o elfTex impedindo outros elfos de entrarem até que todos os três tenham invocado getHelp().



In [ ]:
void Elve() {
    while (true) {
        sem_wait(&elfTex);
        sem_wait(&mutex);
        elves++;
        if (elves == 3)
              sem_post(&santaSem);
        else
            sem_post(&elfTex);
        sem_post(&mutex);

        getHelp();

        sem_wait(&mutex);
        elves--;
        if (elves == 0)
            sem_post(&elfTex);
        sem_post(&mutex);  
    }
}

# Santa problem solution (main)

In [ ]:
%%writefile SantaProblem.c

#include<pthread.h>
#include<semaphore.h>
#include<stdlib.h>
#include<assert.h>
#include<unistd.h>
#include<stdbool.h>
#include<string.h>
#include<stdio.h>
#include<stdint.h>

pthread_t *CreateThread (void *(* f)(void *), void *a){
	pthread_t *t = malloc(sizeof(pthread_t));
	assert(t != NULL);
	int ret = pthread_create(t, NULL, f, a);
	assert(ret == 0);
	return t;
}

// Constantes
static const int NUMBER_REINDEER = 9;
static const int NUMBER_ELVE = 10;

// Contadores
static int elves;
static int reindeer;

// Semáforos
static sem_t santaSem;
static sem_t reindeerSem;

// Mutexes
pthread_mutex_t _mutex;
pthread_mutex_t _elfTex; 

//Implementations of SantaClaus
void *SantaClaus (void *arg){

	printf("Santa Claus\n"); fflush(stdout);

	while(true){
        sem_wait(&santaSem);
        pthread_mutex_lock(&_mutex);
     
        if (reindeer == NUMBER_REINDEER) {
        	printf("Santa Claus: Preparing Sleigh\n"); fflush(stdout);
            
          for(int i=0; i<NUMBER_REINDEER; i++)
                sem_post(&reindeerSem);          
        	printf("Santa Claus: Let's go reindeers\n"); fflush(stdout);
          reindeer = 0;     
        }

        else if (elves == 3) {
        	printf("Santa Claus: Helping elves\n"); fflush(stdout);
        }

        pthread_mutex_unlock(&_mutex);
    }
	
	return arg;
}

//Implementation of Reindeer
void *Reindeer (void *arg){

	int id = (intptr_t) arg;
	printf("This is reindeer %d\n",id); fflush(stdout);

	while (true) {
        pthread_mutex_lock(&_mutex);
        reindeer++;
        pthread_mutex_unlock(&_mutex);

        if (reindeer == NUMBER_REINDEER)
            sem_post(&santaSem);
       
        sem_wait(&reindeerSem);
        printf("Reindeer %d getting hitched\n",id); fflush(stdout);

		    sleep(20);
    }
	
	return arg;
}

//Implementation of Elve
void *Elve (void *arg){
	int id = (intptr_t) arg;
	printf("This is elve %d\n",id); fflush(stdout);

	while (true) {
		bool need_help = random() % 100 < 50;
		if (need_help) {
			  pthread_mutex_lock(&_elfTex);

		    pthread_mutex_lock(&_mutex);
		    elves++;
        pthread_mutex_unlock(&_mutex);

        printf("Elve %d will get help from Santa Claus\n",id); fflush(stdout);

			  if (elves == 3)
              sem_post(&santaSem);
		    else
		        pthread_mutex_unlock(&_elfTex);
		    

			  sleep(10);

			  pthread_mutex_lock(&_mutex);
		    elves--;
        pthread_mutex_unlock(&_mutex); 

		    if (elves == 0)
		        pthread_mutex_unlock(&_elfTex);
 
		}

		printf("Elve %d at work\n",id); fflush(stdout);
		sleep(2 + random() % 5);
	}
	
	return arg;
}

int main(int ac, char **av){
	elves = 0;
	reindeer = 0;
	sem_init(&santaSem, 0, 0);
	sem_init(&reindeerSem, 0, 0);
  pthread_mutex_init(&_mutex, NULL);
  pthread_mutex_init(&_elfTex, NULL);

	
	pthread_t *santa_claus = CreateThread(SantaClaus, 0);
	
	pthread_t *reindeers[NUMBER_REINDEER];
	for(int r = 0; r<NUMBER_REINDEER; r++){
		reindeers[r] = CreateThread(Reindeer, (void *)(intptr_t) r+1);
	}
	
	pthread_t *elves[NUMBER_ELVE];
	for(int e = 0; e<NUMBER_ELVE; e++){
		elves[e] = CreateThread(Elve, (void *)(intptr_t) e+1);
	}
	
	int ret = pthread_join(*santa_claus,NULL);
	assert(ret == 0);
	return 0;
}

Overwriting SantaProblem.c


# Discussão



*   Implementação do Papai noel, Rena e Efo unificadas com prints de forma a demonstrar qual operação está sendo realizada.
*   Foram definidos de forma arbitrária 10 elfos, sendo que a cada chamada há 50% de chance de um elfo precisar de ajuda.
*   Na Main, iniciamos uma thread para o papai noel, uma thread para cada rena e uma thread para cada elfo, de forma que todos ocorrem paralelamente e os semáforos que fazem o controle de cada chamada.

*   Um mutex é usado para garantir a exclusão mútua durante a execução dos loops de Papai Noel, Rena e Elfos. A região crítica é composta pelos contadores de renas e elfos e pelos semáforos de papai noel, elfos e renas.
*   O mutex garante que apenas uma thread acesse e modifique essas variáveis por vez.
*   O mutex ElfTex garante que apenas 3 elfos que tenham pedido ajuda possam sinalizar o semáforo do Papai Noel.
*   O semáforo do Papai Noel pode ser sinalizado pelas duas condições que satisfazem o problema, as chegada das 9 renas ou 3 elfos pedirem ajuda. Em todo começo de loop do método Papai Noel ele é colocado em espera.
*   O problema é proposto de forma que "Se 3 elfos estão esperando e a última rena chegar, o Papai Noel prioriza as renas.". Observe que se as renas continuarem chegando em seguida, pode-se ocorrer uma situação de **starvation** onde o processo dos elfos nunca é executado. Durante a implementação é colocado um comando sleep() dentro da thread de renas para que haja espaço para a thread de elfos ser executada também.

Comportamento esperado:
*   Renas e elfos inicializados. 
*   Quando todas as 9 renas estiverem inicializadas, o papai noel irá chamar o "Preparing Sleigh" e cada uma das renas irá responder com "Getting hitched".
*   Elfos incializados e iniciarão os trabalhos. Há 50% de chance de um elfo precisar de ajuda. Quando 3 deles estiverem precisando de ajuda o papai noel irá chamar o "Helping elves".




In [ ]:
!gcc SantaProblem.c -o SantaProblem -pthread && ./SantaProblem

Santa Claus
This is reindeer 2
This is reindeer 1
This is reindeer 3
This is reindeer 4
This is reindeer 5
This is reindeer 6
This is reindeer 7
This is reindeer 8
This is reindeer 9
Santa Claus: Preparing Sleigh
Santa Claus: Let's go reindeers
Reindeer 2 getting hitched
Reindeer 1 getting hitched
Reindeer 3 getting hitched
Reindeer 4 getting hitched
Reindeer 5 getting hitched
Reindeer 6 getting hitched
Reindeer 7 getting hitched
Reindeer 8 getting hitched
Reindeer 9 getting hitched
This is elve 6
Elve 6 at work
This is elve 7
Elve 7 at work
This is elve 8
Elve 8 at work
This is elve 9
Elve 9 at work
This is elve 10
This is elve 4
Elve 4 will get help from Santa Claus
Elve 10 will get help from Santa Claus
This is elve 2
Elve 2 at work
This is elve 5
Elve 5 at work
This is elve 3
Elve 3 at work
This is elve 1
Elve 1 will get help from Santa Claus
Santa Claus: Helping elves
Elve 8 at work
Elve 3 at work
Elve 2 at work
^C
